In [ ]:
# WIP: Requires LLAMA models.
!git clone https://github.com/ggerganov/llama.cpp \
    && make \

In [7]:
import os

os.environ["CMAKE_ARGS"] = "-DLLAMA_OPENBLAS=on"
os.environ["FORCE_CMAKE"] = "1"

In [8]:
%pip install llama-cpp-python

Note: you may need to restart the kernel to use updated packages.


In [9]:
from llama_cpp import Llama
llm = Llama(model_path="./models/7B/ggml-model.bin")
output = llm("Q: Name the planets in the solar system? A: ", max_tokens=32, stop=["Q:", "\n"], echo=True)
print(output)

ValueError: Model path does not exist: ./models/7B/ggml-model.bin